In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
df.printSchema()

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)



In [0]:
df.show(5)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+---------+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import desc

populated_city=df.select("City").orderBy(desc("2014 Population estimate")).show(10)



+---------------+
|           City|
+---------------+
|    New York[6]|
|    Los Angeles|
|        Chicago|
|     Houston[7]|
|Philadelphia[8]|
|        Phoenix|
|    San Antonio|
|      San Diego|
|         Dallas|
|       San Jose|
+---------------+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import desc

populated_city=df.select("City").orderBy(desc("2014 Population estimate")).limit(10)
display(populated_city)

City
New York[6]
Los Angeles
Chicago
Houston[7]
Philadelphia[8]
Phoenix
San Antonio
San Diego
Dallas
San Jose


In [0]:
  #Average median sales price by state

In [0]:
from pyspark.sql.functions import avg, col

median_sales = df.groupBy("State") \
    .agg(avg(col("2015 median sales price")).alias("Avg Median Price")) \
    .orderBy("Avg Median Price", ascending=False)

median_sales.show()


+--------------------+------------------+
|               State|  Avg Median Price|
+--------------------+------------------+
|             Hawai'i|             699.3|
|          California| 547.9714285714286|
|District of Columbia|             367.8|
|          New Jersey|             350.8|
|            Colorado|333.46666666666664|
|          Washington|266.29999999999995|
|       Massachusetts|             261.8|
|         Connecticut|250.13333333333333|
|                Utah|             243.3|
|       New Hampshire|             237.4|
|              Nevada|             237.3|
|        Rhode Island|             233.3|
|              Oregon|231.53333333333333|
|            Maryland|             223.1|
|           Wisconsin|             210.8|
|           Minnesota|             209.4|
|            Virginia|            201.25|
|             Arizona|             192.1|
|            New York|           185.775|
|        Pennsylvania|             183.8|
+--------------------+------------

In [0]:
#Display city name and population
display_city = df.select("City", "2014 Population estimate")
display_city.show(10)



+-------------+------------------------+
|         City|2014 Population estimate|
+-------------+------------------------+
|   Birmingham|                  212247|
|   Huntsville|                  188226|
|       Mobile|                  194675|
|   Montgomery|                  200481|
|Anchorage[19]|                  301010|
|     Chandler|                  254276|
|  Gilbert[20]|                  239277|
|     Glendale|                  237517|
|         Mesa|                  464704|
|       Peoria|                  166934|
+-------------+------------------------+
only showing top 10 rows



In [0]:
#Display cities with populatin more than 1 million

display_city = df.select("City", "2014 Population estimate").filter(col("2014 Population estimate")>1000000).display()

City,2014 Population estimate
Phoenix,1537058
Los Angeles,3928864
San Diego,1381069
San Jose,1015785
Chicago,2722389
New York[6],8491079
Philadelphia[8],1560297
Dallas,1281047
Houston[7],2239558
San Antonio,1436697


Databricks visualization. Run in Databricks to view.

In [0]:
#Identify cities with missing or zero price values

cities_missing=df.select("City").filter((col("2015 median sales price").isNull()) | (col("2015 median sales price")==0)).limit(20)
cities_missing.display()

City
Anchorage[19]
Chandler
Gilbert[20]
Glendale
Mesa
Peoria
Scottsdale
Surprise
Tempe
Antioch
